In [1]:
import pandas as pd

df = pd.read_csv("/content/Table Servant Data - Sheet1 (3).csv")

In [2]:
print(df.shape)
df.head()

(57, 2)


,INPUT,OUTPUT
0,more to table one,One
1,move to table one,One
2,whew to table,Repeat Again
3,more to cable one,One
4,move to table,Repeat Again


In [3]:
df = df.drop_duplicates()

In [4]:
df.shape

(44, 2)

In [5]:
X = df["               INPUT"]
y = df["              OUTPUT"]

print(y.unique())
from sklearn.preprocessing import LabelEncoder
import numpy as np
y = np.array(y)

encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

['One' 'Repeat Again' 'Two']


In [6]:
y_encoded

array([0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [7]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_words = 2000
max_length = 20

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
X = pad_sequences(sequences, maxlen=max_length)

In [8]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
y_one_hot = encoder.fit_transform(y_encoded.reshape(-1, 1)).toarray()


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM

model = Sequential()
model.add(Embedding(max_words, 32, input_length=max_length))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(128))
model.add(Dense(128, activation='linear'))
model.add(Dense(256, activation='linear'))
model.add(Dense(512, activation='linear'))
model.add(Dense(128, activation='linear'))
model.add(Dense(3, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 32)            64000     
                                                                 
 lstm (LSTM)                 (None, 20, 64)            24832     
                                                                 
 lstm_1 (LSTM)               (None, 128)               98816     
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 512)               131584    
                                                                 
 dense_3 (Dense)             (None, 128)               6

In [10]:
# X = X.toarray()
hist = model.fit(X, y_encoded, validation_split=0.2, epochs=5, batch_size=20)

Epoch 1/5
2/2 [==============================] - 12s 1s/step - loss: 1.0543 - accuracy: 0.7429 - val_loss: 1.2749 - val_accuracy: 0.0000e+00
Epoch 2/5
2/2 [==============================] - 0s 68ms/step - loss: 0.8661 - accuracy: 0.7429 - val_loss: 2.0150 - val_accuracy: 0.0000e+00
Epoch 3/5
2/2 [==============================] - 0s 82ms/step - loss: 0.7460 - accuracy: 0.7429 - val_loss: 1.7461 - val_accuracy: 0.0000e+00
Epoch 4/5
2/2 [==============================] - 0s 83ms/step - loss: 0.7016 - accuracy: 0.7429 - val_loss: 1.3959 - val_accuracy: 0.0000e+00
Epoch 5/5
2/2 [==============================] - 0s 82ms/step - loss: 0.7106 - accuracy: 0.7429 - val_loss: 1.2793 - val_accuracy: 0.0000e+00


In [11]:
cleaned_text = "move to table one"
sequence = tokenizer.texts_to_sequences([cleaned_text])
padded_sequence = pad_sequences(sequence, maxlen=max_length)
np.argmax(model.predict(padded_sequence))

1/1 [==============================] - 1s 1s/step


0